In [9]:
import sqlite3
import pandas as pd
import numpy as np

In [10]:
#### le arquivo CSV e cria DataFrame "df" ####

df = pd.read_csv('C:/Users/Gustavo Calanzani/Documents/Projeto Integrador 3/dados.csv')

# Exclui linhas com valores faltantes
df.dropna(inplace=True)

# Exclui linhas com valores inválidos
condicaoNotasInvalidas = ((df['math score'] > 100) | (df['math score'] < 0) | 
                         (df['reading score'] > 100) | (df['reading score'] < 0) | 
                         (df['writing score'] > 100) | (df['writing score'] < 0))

df.drop(df.loc[condicaoNotasInvalidas].index, inplace=True)

# Mostra dataframe
df

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group D,some college,standard,completed,59,70,78
1,male,group D,associate's degree,standard,none,96,93,87
2,female,group D,some college,free/reduced,none,57,76,77
3,male,group B,some college,free/reduced,none,70,70,63
4,female,group D,associate's degree,standard,none,83,85,86
...,...,...,...,...,...,...,...,...
995,male,group C,some college,standard,none,77,77,71
996,male,group C,some college,standard,none,80,66,66
997,female,group A,high school,standard,completed,67,86,86
998,male,group E,high school,standard,none,80,72,62


In [11]:
### Cria novo DataFrame "dfFato", cópia de "df" mas apenas com os campos de notas ###

dfFato = df[['math score', 'reading score', 'writing score']].copy()
dfFato.index.name = 'ID'
dfFato

,math score,reading score,writing score
ID,,,
0,59,70,78
1,96,93,87
2,57,76,77
3,70,70,63
4,83,85,86
...,...,...,...
995,77,77,71
996,80,66,66
997,67,86,86


In [12]:
########### Cria tabela fato "tabela_fato" no SQLite ###########

# Criar a conexão com o banco de dados
conn = sqlite3.connect('bancodedados.db')

# Carregar o dataframe para o banco de dados como uma tabela fato
dfFato.to_sql('tabela_fato', conn, if_exists='replace', index=False)

# Fecha a conexão com o banco de dados
conn.close()

In [13]:
########### Visualizar tabela fato ###########

# Criar a conexão com o banco de dados
conn = sqlite3.connect('bancodedados.db')

# Executar a consulta SQL para selecionar todos os dados da tabela fato
dfFato_sqlite = pd.read_sql_query("SELECT * from tabela_fato", conn)

# Fechar a conexão com o banco de dados
conn.close()

# Visualizar o dataframe
dfFato_sqlite

,math score,reading score,writing score
0,59,70,78
1,96,93,87
2,57,76,77
3,70,70,63
4,83,85,86
...,...,...,...
995,77,77,71
996,80,66,66
997,67,86,86
998,80,72,62


In [14]:
########### Cria tabela_dimensao ###########

conn = sqlite3.connect('bancodedados.db')
cursor = conn.cursor()

#Apaga a tabela dimensão se ela já existe
cursor.execute('DROP TABLE IF EXISTS tabela_dimensao')

#Cria a tabela dimensão
cursor.execute('''
    CREATE TABLE tabela_dimensao (
        id INTEGER PRIMARY KEY,
        gender TEXT,
        race_ethnicity TEXT,
        parental_level_of_education TEXT,
        lunch TEXT,
        test_preparation_course TEXT
    )
''')

# insere os dados na tabela dimensão, usando o índice da tabela fato como chave primária
for i, row in df.iterrows():
    query = f"""
        INSERT INTO tabela_dimensao (id, gender, race_ethnicity, parental_level_of_education, lunch, test_preparation_course)
        VALUES ({i}, "{row['gender']}", "{row['race/ethnicity']}", "{row['parental level of education']}", "{row['lunch']}", "{row['test preparation course']}")
    """
    cursor.execute(query)

#Commit das mudanças e fechamento da conexão
conn.commit()
conn.close()


In [15]:
########### Visualizar tabela dimensao ###########

# Criar a conexão com o banco de dados
conn = sqlite3.connect('bancodedados.db')

# Executar a consulta SQL para selecionar todos os dados da tabela fato
dfDim_sqlite = pd.read_sql_query("SELECT * from tabela_dimensao", conn)

# Fechar a conexão com o banco de dados
conn.close()

# Visualizar o dataframe
dfDim_sqlite

,id,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course
0,0,female,group D,some college,standard,completed
1,1,male,group D,associate's degree,standard,none
2,2,female,group D,some college,free/reduced,none
3,3,male,group B,some college,free/reduced,none
4,4,female,group D,associate's degree,standard,none
...,...,...,...,...,...,...
995,995,male,group C,some college,standard,none
996,996,male,group C,some college,standard,none
997,997,female,group A,high school,standard,completed
998,998,male,group E,high school,standard,none
